In [1]:
%run 01_weather_api.ipynb

Dates        : ['2026-02-25', '2026-02-26', '2026-02-27', '2026-02-28', '2026-03-01', '2026-03-02', '2026-03-03']
Temp max (°C): [10.5, 9.5, 9.5, 7.5, 5.8, 11.4, 11.3]
Temp min (°C): [4.4, 3.3, 2.3, 1.7, 0.3, 1.6, 4.9]
Précip (mm)  : [0.0, 0.0, 0.0, 0.0, 0.0, 0.3, 0.0]
Neige (cm)   : [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
Vent (km/h)  : [10.3, 10.4, 10.7, 6.5, 7.9, 11.7, 10.0]


In [5]:
import anthropic
import json
from dotenv import load_dotenv

load_dotenv()
client = anthropic.Anthropic()

# Définition du tool
weather_tool = {
    "name": "get_weather",
    "description": "Récupère les prévisions météo et enneigement pour une station de ski sur 7 jours.",
    "input_schema": {
        "type": "object",
        "properties": {
            "latitude": {
                "type": "number", 
                "description": "Latitude de la station de ski."
            },
            "longitude": {
                "type": "number", 
                "description": "Longitude de la station de ski."
            },
            "station_name": {
                "type": "string", 
                "description": "Nom de la station de ski."
            }
        },
        "required": ["latitude", "longitude", "station_name"]
    }
}

# Execution du tool
def execute_tool(tool_name: str, tool_input: dict):
    if tool_name == "get_weather":
        result = get_weather(**tool_input)
        return json.dumps(result, ensure_ascii=False)
    else:
        raise ValueError(f"Tool {tool_name} non reconnu.")
    
def ask_with_tools(user_message: str):
    messages = [{"role": "user", "content": user_message}]
    
    while True:
        response = client.messages.create(
            model="claude-sonnet-4-6",
            messages=messages,
            tools=[weather_tool],
            max_tokens=4096
        )
        
        if response.stop_reason == "end_turn":
            return response.content[0].text
        
        if response.stop_reason == "tool_use":
            messages.append({"role": "assistant", "content": response.content})
            
            tool_results = []
            for block in response.content:
                if block.type == "tool_use":
                    print(f"Utilisation de l'outil : {block.name} avec {block.input}")
                    result = execute_tool(block.name, block.input)
                    tool_results.append({
                        "type": "tool_result",
                        "tool_use_id": block.id,
                        "content": result
                    })
                    
            # Ajouter les résultats des outils aux messages pour que le modèle puisse les utiliser dans la suite de la conversation
            messages.append({"role": "user", "content": tool_results})
            
# Test
response = ask_with_tools(
    "Quelle est la prévision d'enneigement pour Les Orres cette semaine ?"
)
print(response)

Utilisation de l'outil : get_weather avec {'latitude': 44.5167, 'longitude': 6.5333, 'station_name': 'Les Orres'}
Voici les prévisions d'enneigement pour la station des **Orres** (altitude ~1310 m) pour cette semaine :

---

### ❄️ Résumé de l'enneigement – Les Orres (25 fév. – 3 mars 2026)

| Jour | Épaisseur de neige | Chutes de neige | Temp. min / max | Vent max |
|---|---|---|---|---|
| 📅 Mer. 25 fév. | ~32 cm | 0 cm | 6° / 13°C | 12,5 km/h |
| 📅 Jeu. 26 fév. | ~28 cm | 0 cm | 5° / 12°C | 12,5 km/h |
| 📅 Ven. 27 fév. | ~22 cm | 0 cm | 3° / 11°C | 11,5 km/h |
| 📅 Sam. 28 fév. | ~16 cm | 0 cm | 1° / 9°C | 8,4 km/h |
| 📅 Dim. 1ᵉʳ mars | ~10 cm | 0 cm | 0° / 8°C | 6,9 km/h |
| 📅 Lun. 2 mars | ~10 cm | 0 cm | 2° / 14°C | 11,9 km/h |
| 📅 Mar. 3 mars | ~3 cm | 0 cm | 7° / 14°C | 10,0 km/h |

---

### 🔍 Analyse

- 🌨️ **Aucune chute de neige prévue** sur les 7 prochains jours.
- 📉 **L'enneigement est en nette diminution** : le manteau neigeux passe de **32 cm à seulement ~3 cm** en fin de s